## 20 news groups
Задача: определить по тексту поста, к какой теме (из 20 возможных) он относится 

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
newsgroups = fetch_20newsgroups()

In [4]:
# не знаем, что за объект возвраащет fetch_20newsgroups() и какие у него атрибуты
# смотрим
dir(newsgroups)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [5]:
print(newsgroups.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality 

Посмотрим на названия тем:

In [6]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Посмотрим на сами тексты:

In [7]:
newsgroups.data[:10]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [8]:
df = pd.DataFrame()
df['text'] = newsgroups.data
df['target'] = newsgroups.target

In [9]:
df.head(10)

,text,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14
5,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,16
6,From: bmdelane@quads.uchicago.edu (brian manni...,13
7,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,3
8,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,2
9,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,4


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
SEED = 123
np.random.seed(123)
df_train, df_test = train_test_split(df, train_size=0.5, test_size=0.3, stratify=df.target)

In [12]:
df_train.shape

(5657, 2)

# Преобразовать текстовые данные в признаки 

## 1. Bag of words

Каждый документ - мешок слов. Не важно в каком они порядке, не важно как они связаны между собой, вообще ничего не важно кроме того, какие именно слова есть в документе и в каком количестве. 

------------------------------------
![bow](http://uc-r.github.io/public/images/analytics/feature-engineering/bow-image.png)
--------------------------------------

Процесс преобразования:
1. Токенизация 
2. (Предобработка)
3. Построение словаря по всем документам коллекции
4. Вектор признаков для кажого документа - количество вхождений каждого слова из словаря в документ

----------------------------------------
![bow2](https://raw.githubusercontent.com/Yorko/mlcourse.ai/3380f07cebee0b44a6cc9e1d4e36eb4675ae37fd/img/bag_of_words.png)

---------------------------------
Итоговая матрица признаков получается 
+ очень большой (***Кол-во объектов*** х ***Размер словаря***) 
+ очень разреженной (много 0). 

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

`sklearn.feature_extraction.text.CountVectorizer`
+ ***fit*** - строит словарь по коллекции текстов
+ ***transform*** - преобразует коллекцию текстов в матрицу признаков по полученному после ***fit*** словарю 
+ по умолчанию в качестве элемента словаря выступают отдельные слова

In [14]:
vectorizer = CountVectorizer()

In [15]:
%%time
X_train = vectorizer.fit_transform(df_train.text) # постоить словарь по обучающей выборке и преобразовать ее в матрицу
X_test = vectorizer.transform(df_test.text) # преобразовать тестовую выборку в матрицу

CPU times: user 2.13 s, sys: 10.8 ms, total: 2.14 s
Wall time: 2.22 s


In [16]:
print('Размер словаря: %s элементов' % X_train.shape[1])

Размер словаря: 85615 элементов


In [17]:
# посмотреть на получившийся словарь
vectorizer.vocabulary_

{'from': 36822,
 'nrmendel': 57333,
 'unix': 78596,
 'amherst': 16431,
 'edu': 32315,
 'nathaniel': 56003,
 'mendell': 52699,
 'subject': 73314,
 're': 65114,
 'opinions': 58624,
 'wanted': 81347,
 'help': 40656,
 'nntp': 56952,
 'posting': 62063,
 'host': 41582,
 'amhux3': 16433,
 'organization': 58817,
 'college': 25539,
 'newsreader': 56532,
 'tin': 76035,
 'version': 80071,
 'pl7': 61377,
 'lines': 49392,
 'what': 81899,
 'size': 70733,
 'dirtbikes': 30181,
 'did': 29962,
 'you': 84600,
 'ride': 66777,
 'and': 16647,
 'for': 36322,
 'how': 41650,
 'long': 49844,
 'might': 53289,
 'be': 19547,
 'able': 14687,
 'to': 76196,
 'slip': 71013,
 'into': 44229,
 '500cc': 8545,
 'bike': 20283,
 'like': 49295,
 'keep': 46838,
 'telling': 75175,
 'people': 60521,
 'though': 75765,
 'buy': 22077,
 'an': 16557,
 'older': 58337,
 'cheaper': 24182,
 'that': 75513,
 'while': 81951,
 'first': 35815,
 '500': 8534,
 'interceptor': 44055,
 'as': 17697,
 'example': 34160,
 'zx': 85566,
 '10': 1208,
 'd

In [18]:
# по умолчанию вся пунктуция удаляется
'!' in vectorizer.vocabulary_

False

### scipy.sparse 

Результатом метода ***transform*** будет разреженная матрица - объект  `csr_matrix`.    
Такие объекты оптимизируют работу с разреженными данными (в памяти хранятся **только ненулевые значения**). 
+ (+) занимает маньше места в памяти, чем те же данные в другом формате (например `pd.DataFrame` или `numpy.ndarray`)
+ (+) модели учатся быстрее, чем на тех же данных в другом формате 
+ (-) не все модели из `sklearn` работают с разреженными данными
+ (-) нельзя так же быстро и легко посмотреть названия признаков как в `pd.DataFrame`

In [19]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [20]:
%%time
# можно преобразовать в неразреженный вид например так (может занять довольно много времени для большой матрицы)
X_train_array = X_train.toarray()

CPU times: user 188 ms, sys: 326 ms, total: 514 ms
Wall time: 513 ms


In [21]:
type(X_train_array)

numpy.ndarray

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, random_state=SEED)

In [23]:
%%time
rf.fit(X_train, df_train.target) # быстро

CPU times: user 6.51 s, sys: 42.1 ms, total: 6.55 s
Wall time: 5.93 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [24]:
y_pred = rf.predict(X_test)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.782     0.771     0.776       144
           1      0.648     0.726     0.685       175
           2      0.639     0.819     0.718       177
           3      0.565     0.661     0.609       177
           4      0.772     0.741     0.757       174
           5      0.828     0.809     0.818       178
           6      0.690     0.847     0.760       176
           7      0.844     0.792     0.817       178
           8      0.866     0.933     0.898       180
           9      0.863     0.878     0.871       180
          10      0.913     0.933     0.923       180
          11      0.877     0.961     0.917       178
          12      0.777     0.492     0.602       177
          13      0.908     0.719     0.803       178
          14      0.893     0.888     0.890       178
          15      0.702     0.917     0.795       180
          16      0.921     0.921     0.921       164
          17      0.936    

In [25]:
%%time 
rf.fit(X_train_array, df_train.target) # ~ в 10 раз дольше

CPU times: user 50 s, sys: 311 ms, total: 50.3 s
Wall time: 50.3 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [26]:
# качество абсолютно одинаковое, т.к. данные одни и те же 
y_pred = rf.predict(X_test.toarray())
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.782     0.771     0.776       144
           1      0.648     0.726     0.685       175
           2      0.639     0.819     0.718       177
           3      0.565     0.661     0.609       177
           4      0.772     0.741     0.757       174
           5      0.828     0.809     0.818       178
           6      0.690     0.847     0.760       176
           7      0.844     0.792     0.817       178
           8      0.866     0.933     0.898       180
           9      0.863     0.878     0.871       180
          10      0.913     0.933     0.923       180
          11      0.877     0.961     0.917       178
          12      0.777     0.492     0.602       177
          13      0.908     0.719     0.803       178
          14      0.893     0.888     0.890       178
          15      0.702     0.917     0.795       180
          16      0.921     0.921     0.921       164
          17      0.936    

### N-grams

Помимо отдельных слов при векторизации можно использовать н-граммы. 
+ (+) возможно качество модели будет лучше, тк информации больше
+ (-) словарь, а с ним и матрица признаков становится сильно больше 

In [27]:
# отдельные слова, биграммы, триграммы
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3))

In [28]:
%%time
X_train_ngram = ngram_vectorizer.fit_transform(df_train.text) 
X_test_ngram = ngram_vectorizer.transform(df_test.text)

CPU times: user 12 s, sys: 252 ms, total: 12.3 s
Wall time: 12.3 s


In [29]:
print('Размер словаря: %s элементов' % X_train_ngram.shape[1])

Размер словаря: 1816567 элементов


In [30]:
%%time
rf.fit(X_train_ngram, df_train.target)

CPU times: user 54 s, sys: 19.9 ms, total: 54 s
Wall time: 53 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [31]:
# действительно, качество стало получше
y_pred = rf.predict(X_test_ngram)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.949     0.771     0.851       144
           1      0.534     0.754     0.626       175
           2      0.623     0.729     0.672       177
           3      0.595     0.621     0.608       177
           4      0.809     0.730     0.767       174
           5      0.819     0.736     0.775       178
           6      0.594     0.881     0.709       176
           7      0.841     0.831     0.836       178
           8      0.827     0.906     0.865       180
           9      0.825     0.839     0.832       180
          10      0.897     0.917     0.907       180
          11      0.933     0.938     0.936       178
          12      0.778     0.554     0.647       177
          13      0.913     0.708     0.797       178
          14      0.968     0.860     0.911       178
          15      0.688     0.906     0.782       180
          16      0.933     0.939     0.936       164
          17      0.993    

### Уменьшить размер словаря

Параметры
+ **min_df: *float in range [0.0, 1.0] or int, default=1*** - при построении словаря не учитывать элементы, которые встречаются в количестве документов меньше заданного порога.     
Полезно, если мы не хотим учитывать в словаре всякие странные слова (с опечатками, имена собственные,и т.д.), которые встречаются в очень малом количестве документов и из-за этого неинформативны для модели (почти константный признак, состоящий в основном из 0.) 
+ **max_df: *float in range [0.0, 1.0] or int, default=1.0*** - при построении словаря не учитывать элементы, которые встречаются в количестве документов больше заданного порога.     
Полезно, если мы не хотим учитывать в словаре очень частотные слова (союзы, местоимения), которые встречаются во всех или почти во всех документах и из-за этого неинформативны для модели (почти константный признак). 
+ **max_features: *int, default=None*** - эксплицитно ограничить размер словаря заданным числом, в словарь войдет заданное число самых частотных элементов. 

#### min_df & max_df

In [32]:
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3), min_df=5, max_df=0.7)

In [33]:
X_train_ngram_trunc = ngram_vectorizer.fit_transform(df_train.text)
X_test_ngram_trunc = ngram_vectorizer.transform(df_test.text)

In [34]:
print('Размер словаря до: %s элементов' % X_train_ngram.shape[1])
print('Размер словаря с ограничениями min_df и max_df: %s элементов' % X_train_ngram_trunc.shape[1])

Размер словаря до: 1816567 элементов
Размер словаря с ограничениями min_df и max_df: 80469 элементов


In [35]:
%%time
rf.fit(X_train_ngram_trunc, df_train.target) 

CPU times: user 6.19 s, sys: 19.9 ms, total: 6.21 s
Wall time: 5.51 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [36]:
# качество не пострадало, даже улучшилось 
# обучается намного быстрее
y_pred = rf.predict(X_test_ngram_trunc)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.931     0.847     0.887       144
           1      0.702     0.714     0.708       175
           2      0.603     0.842     0.703       177
           3      0.559     0.593     0.575       177
           4      0.764     0.764     0.764       174
           5      0.812     0.803     0.808       178
           6      0.748     0.858     0.799       176
           7      0.817     0.826     0.821       178
           8      0.885     0.894     0.890       180
           9      0.817     0.867     0.841       180
          10      0.874     0.883     0.878       180
          11      0.919     0.955     0.937       178
          12      0.723     0.559     0.631       177
          13      0.895     0.719     0.798       178
          14      0.910     0.910     0.910       178
          15      0.744     0.922     0.824       180
          16      0.928     0.939     0.933       164
          17      0.964    

#### max_features

In [37]:
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=10000)

In [38]:
X_train_ngram_trunc = ngram_vectorizer.fit_transform(df_train.text)
X_test_ngram_trunc = ngram_vectorizer.transform(df_test.text)

In [39]:
print('Размер словаря до: %s элементов' % X_train_ngram.shape[1])
print('Размер словаря с ограничениями max_features: %s элементов' % X_train_ngram_trunc.shape[1])

Размер словаря до: 1816567 элементов
Размер словаря с ограничениями max_features: 10000 элементов


In [40]:
%%time
rf.fit(X_train_ngram_trunc, df_train.target)

CPU times: user 4.63 s, sys: 15.9 ms, total: 4.65 s
Wall time: 4.01 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [41]:
# обучилось быстро, но качество пострадало
y_pred = rf.predict(X_test_ngram_trunc)
print(classification_report(df_test.target, y_pred, digits=3)) 

              precision    recall  f1-score   support

           0      0.869     0.785     0.825       144
           1      0.638     0.646     0.642       175
           2      0.652     0.751     0.698       177
           3      0.522     0.605     0.560       177
           4      0.721     0.713     0.717       174
           5      0.802     0.775     0.789       178
           6      0.717     0.835     0.772       176
           7      0.787     0.770     0.778       178
           8      0.886     0.906     0.896       180
           9      0.727     0.872     0.793       180
          10      0.915     0.839     0.875       180
          11      0.902     0.933     0.917       178
          12      0.611     0.514     0.558       177
          13      0.840     0.736     0.784       178
          14      0.875     0.865     0.870       178
          15      0.736     0.900     0.810       180
          16      0.890     0.933     0.911       164
          17      0.934    

#### Предобработка
+ Лемматизация или стемминг (меньше разных слов = меньше словарь)
+ Стоп-слова 
+ И то, и друое можно сделать сразу в объекте векторизатора, ередав нужные параметры
    + **preprocessor: *callable, default=None*** - вызываемый объект, который будет делать предобработку, должен принимать на вход слвоо и возвращать его же  в обработанном виде 
    + **stop_words: *list, default=None*** - список стоп-слов

In [42]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 


In [43]:
eng_stop_words = stopwords.words('english')
print(eng_stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [44]:
lemmatizer = WordNetLemmatizer() 

In [45]:
vectorizer_preproc = CountVectorizer(ngram_range=(1, 3), stop_words=eng_stop_words, preprocessor=lemmatizer.lemmatize)

In [46]:
%%time
X_train_preproc = vectorizer_preproc.fit_transform(df_train.text)
X_test_preproc = vectorizer_preproc.transform(df_test.text)

CPU times: user 11 s, sys: 200 ms, total: 11.2 s
Wall time: 11.4 s


In [47]:
print('Размер словаря до: %s элементов' % X_train_ngram.shape[1])
print('Размер словаря после предобрабоки: %s элементов' % X_train_preproc.shape[1])

Размер словаря до: 1816567 элементов
Размер словаря после предобрабоки: 1607473 элементов


In [48]:
%%time
rf.fit(X_train_preproc, df_train.target)

CPU times: user 48.9 s, sys: 27.8 ms, total: 48.9 s
Wall time: 48 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [49]:
y_pred = rf.predict(X_test_preproc)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.898     0.854     0.875       144
           1      0.525     0.783     0.628       175
           2      0.648     0.802     0.717       177
           3      0.645     0.718     0.679       177
           4      0.831     0.678     0.747       174
           5      0.856     0.770     0.811       178
           6      0.592     0.875     0.706       176
           7      0.891     0.826     0.857       178
           8      0.908     0.928     0.918       180
           9      0.824     0.856     0.839       180
          10      0.856     0.894     0.875       180
          11      0.965     0.927     0.946       178
          12      0.827     0.514     0.634       177
          13      0.927     0.713     0.806       178
          14      0.940     0.876     0.907       178
          15      0.753     0.900     0.820       180
          16      0.956     0.927     0.941       164
          17      0.969    

### Char 

+ В качестве элемента словаря можно взять не целые слова, а символы/символьные н-граммы, за это отвечает параметр **analyzer**
+ **analyzer: *{‘word’, ‘char’, ‘char_wb’} or callable, default=’word’***
    + word - слова
    + char - символы
    + char_wb -  символы, но н-граммы создаются только в пределах границ слова

In [50]:
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 5))

In [51]:
%%time
X_train_char = char_vectorizer.fit_transform(df_train.text)
X_test_char = char_vectorizer.transform(df_test.text)

CPU times: user 54.4 s, sys: 432 ms, total: 54.8 s
Wall time: 54.8 s


In [52]:
print('Размер словаря: %s элементов' % X_train_char.shape[1])

Размер словаря: 1686613 элементов


In [53]:
char_vectorizer.vocabulary_

{'fr': 1002451,
 'ro': 1430522,
 'om': 1323682,
 'm:': 1214328,
 ': ': 595486,
 ' n': 103009,
 'nr': 1287283,
 'rm': 1428055,
 'me': 1225258,
 'en': 956543,
 'nd': 1267486,
 'de': 893904,
 'el': 951284,
 'l@': 1170409,
 '@u': 695931,
 'un': 1569658,
 'ni': 1278631,
 'ix': 1118119,
 'x.': 1628799,
 '.a': 335120,
 'am': 782666,
 'mh': 1229222,
 'he': 1052129,
 'er': 962646,
 'rs': 1435320,
 'st': 1491024,
 't.': 1510570,
 '.e': 338094,
 'ed': 940706,
 'du': 907253,
 'u ': 1547927,
 ' (': 40326,
 '(n': 221797,
 'na': 1263706,
 'at': 800008,
 'th': 1527162,
 'ha': 1048897,
 'an': 785109,
 'ie': 1088250,
 'l ': 1157978,
 ' m': 100353,
 'll': 1186704,
 'l)': 1163358,
 ')\n': 226441,
 '\ns': 30017,
 'su': 1494355,
 'ub': 1558620,
 'bj': 825970,
 'je': 1127512,
 'ec': 938952,
 'ct': 867765,
 't:': 1514665,
 ' r': 109142,
 're': 1415775,
 'e:': 931776,
 ' o': 104781,
 'op': 1333212,
 'pi': 1367544,
 'in': 1099515,
 'io': 1104677,
 'on': 1326430,
 'ns': 1287734,
 's ': 1446917,
 ' w': 118596,
 '

In [54]:
%%time
rf.fit(X_train_char, df_train.target)

CPU times: user 42.7 s, sys: 124 ms, total: 42.8 s
Wall time: 41.6 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [55]:
y_pred = rf.predict(X_test_char)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.874     0.771     0.819       144
           1      0.620     0.634     0.627       175
           2      0.586     0.791     0.673       177
           3      0.521     0.571     0.544       177
           4      0.693     0.701     0.697       174
           5      0.825     0.792     0.808       178
           6      0.700     0.824     0.757       176
           7      0.736     0.753     0.744       178
           8      0.870     0.894     0.882       180
           9      0.770     0.856     0.811       180
          10      0.894     0.889     0.891       180
          11      0.884     0.938     0.910       178
          12      0.649     0.480     0.552       177
          13      0.853     0.685     0.760       178
          14      0.846     0.865     0.856       178
          15      0.706     0.867     0.778       180
          16      0.887     0.909     0.898       164
          17      0.940    

## 2. Tf-idf

**Term frequency - inverted document frequency**     
Формула:

![](https://miro.medium.com/max/1200/1*V9ac4hLVyms79jl65Ym_Bw.jpeg)

+ **TF** - оценивает важность слова для конкретного документа, (чем чаще встречается это слово в документе, тем больше значение)
+ **IDF** - инверсия частоты, с которой некоторое слово встречается в документах коллекции. Чем чаще слово встречается во всей коллекции, тем меньше значение IDF. 
+ В итоге большой вес получают специфичные для конкретного документа слова (часто встречаются в документе, редко встречаются в остальной коллекции). 
+ В отличие от CountVectorizer учиывает не только информацию о конкретном документе, но и о всей коллеции в целом. 
+ Можно делать все те же вещи, что и c CountVectorizer (параметры одинаковые). 

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 4), min_df=5, max_df=0.7)

In [58]:
%%time
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train.text)
X_test_tfidf = tfidf_vectorizer.transform(df_test.text)

CPU times: user 19.9 s, sys: 484 ms, total: 20.4 s
Wall time: 19.3 s


In [59]:
%%time
rf.fit(X_train_tfidf, df_train.target)

CPU times: user 9.03 s, sys: 20 ms, total: 9.05 s
Wall time: 8.37 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [60]:
y_pred = rf.predict(X_test_tfidf)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.944     0.819     0.877       144
           1      0.688     0.669     0.678       175
           2      0.637     0.814     0.715       177
           3      0.551     0.644     0.594       177
           4      0.808     0.724     0.764       174
           5      0.763     0.798     0.780       178
           6      0.733     0.875     0.798       176
           7      0.835     0.826     0.831       178
           8      0.878     0.917     0.897       180
           9      0.827     0.850     0.838       180
          10      0.878     0.917     0.897       180
          11      0.938     0.938     0.938       178
          12      0.653     0.542     0.593       177
          13      0.896     0.775     0.831       178
          14      0.873     0.888     0.880       178
          15      0.729     0.911     0.810       180
          16      0.905     0.933     0.919       164
          17      0.963    

## Best Features

In [61]:
# вытащить названия признаков из векторизатора
feature_names = tfidf_vectorizer.get_feature_names()

In [62]:
feature_names[:10]

['00',
 '00 00',
 '00 00 00',
 '00 00 00 gmt',
 '00 00 gmt',
 '00 00 gmt lines',
 '00 00 gmt organization',
 '00 07',
 '00 07 gmt',
 '00 07 gmt reply']

In [63]:
sorted_features = sorted(zip(feature_names, rf.feature_importances_), reverse=True, key=lambda x: x[1])

In [64]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [65]:
sorted_features[:30]

[('dod', 0.006441184314773789),
 ('for sale', 0.005390643992840877),
 ('windows', 0.005257964573445574),
 ('bike', 0.00452074195426266),
 ('sale', 0.0041371818412049705),
 ('car', 0.004047726334597508),
 ('mac', 0.003930976085093516),
 ('space', 0.0035887218158681487),
 ('baseball', 0.0033159988150783517),
 ('gun', 0.0030451133799665664),
 ('cars', 0.002989307159215539),
 ('israel', 0.002867347302963976),
 ('god', 0.0028107020479687227),
 ('hockey', 0.0025927652194887314),
 ('team', 0.0023736439517400734),
 ('encryption', 0.00228747784570058),
 ('subject re', 0.0022257540712008667),
 ('guns', 0.0022072283607178678),
 ('orbit', 0.0021660268258747507),
 ('the clipper', 0.0019477688850169802),
 ('apple', 0.0019386342039969498),
 ('shipping', 0.001796141501072326),
 ('graphics', 0.0017454704681345394),
 ('key', 0.0017291293454303963),
 ('christ', 0.0017076417084525663),
 ('game', 0.0017074195244460239),
 ('article apr', 0.0016953326552558569),
 ('israeli', 0.0016909472665551327),
 ('window

## PoS

+ векторизировать имеет смысл не только сами слова, но и разную другую информацию о них
+ например части речи

In [66]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag

In [67]:
pos_tag(word_tokenize('I love cats!!!'))

[('I', 'PRP'),
 ('love', 'VBP'),
 ('cats', 'NNS'),
 ('!', '.'),
 ('!', '.'),
 ('!', '.')]

In [68]:
# определяем части речи слов и склеиваем в строку для дальнейшей векторизации
def get_pos_str(text):
    pos = ' '.join([an[1] for an in pos_tag(word_tokenize(text)) if an[1] != '.'])
    return pos

In [69]:
get_pos_str('I love cats!!!')

'PRP VBP NNS'

In [70]:
%%time
df_train['pos'] = [get_pos_str(text) for text in df_train.text]
df_test['pos'] = [get_pos_str(text) for text in df_test.text]

CPU times: user 2min 28s, sys: 148 ms, total: 2min 28s
Wall time: 2min 28s


In [71]:
vectorizer = CountVectorizer()

In [72]:
%%time
X_train_pos = vectorizer.fit_transform(df_train.pos)
X_test_pos = vectorizer.transform(df_test.pos)

CPU times: user 1.31 s, sys: 3.98 ms, total: 1.32 s
Wall time: 1.32 s


In [73]:
vectorizer.vocabulary_

{'in': 5,
 'nn': 11,
 'nnp': 12,
 'vbd': 26,
 'jj': 6,
 'cd': 1,
 'nnps': 13,
 'wp': 32,
 'nns': 14,
 'prp': 17,
 'vb': 25,
 'cc': 0,
 'wrb': 33,
 'md': 10,
 'to': 23,
 'dt': 2,
 'vbp': 29,
 'vbg': 27,
 'jjr': 7,
 'rb': 18,
 'vbz': 30,
 'vbn': 28,
 'rp': 21,
 'wdt': 31,
 'pos': 16,
 'rbr': 19,
 'jjs': 8,
 'fw': 4,
 'ex': 3,
 'uh': 24,
 'rbs': 20,
 'pdt': 15,
 'sym': 22,
 'ls': 9}

Обучаться только на векторах частей речи скорее всего будет неэффективно, поэтому давайте присоединим векторы частей речи к векторам слов, полученным ранее. 

In [74]:
from scipy.sparse import hstack, vstack

In [75]:
X_train = hstack((X_train_pos, X_train_tfidf)) # горизонтальное соединение
X_test = hstack((X_test_pos, X_test_tfidf))

In [76]:
# проверяем, что мы все правильно соединили, число объектов должно остаться прежним
X_train.shape[0] == X_train_tfidf.shape[0]

True

In [77]:
%%time
rf.fit(X_train, df_train.target)

CPU times: user 6.91 s, sys: 3.98 ms, total: 6.91 s
Wall time: 6.22 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [78]:
# качество стало немного лучше
y_pred = rf.predict(X_test)
print(classification_report(df_test.target, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.925     0.854     0.888       144
           1      0.650     0.754     0.698       175
           2      0.688     0.774     0.729       177
           3      0.631     0.695     0.661       177
           4      0.801     0.741     0.770       174
           5      0.824     0.787     0.805       178
           6      0.703     0.875     0.780       176
           7      0.829     0.843     0.836       178
           8      0.869     0.883     0.876       180
           9      0.781     0.811     0.796       180
          10      0.848     0.867     0.857       180
          11      0.918     0.949     0.934       178
          12      0.680     0.565     0.617       177
          13      0.897     0.736     0.809       178
          14      0.859     0.893     0.876       178
          15      0.724     0.917     0.809       180
          16      0.931     0.909     0.920       164
          17      0.981    

**Задание**
+ вывести наиболее важные признаки этой модели 

In [79]:
feature_names_pos = vectorizer.get_feature_names()
feature_names_tfidf = tfidf_vectorizer.get_feature_names()

In [80]:
# обращаем внимание на порядок, в котором мы соединяли матрицы признаков
feature_names_pos.extend(feature_names_tfidf)

In [81]:
sorted_features = sorted(zip(feature_names_pos, rf.feature_importances_), reverse=True, key=lambda x: x[1])

In [82]:
sorted_features[:50]

[('for sale', 0.006316028509411893),
 ('car', 0.00468182834400963),
 ('windows', 0.004574634474950005),
 ('dod', 0.004407762714555359),
 ('bike', 0.004386785523425288),
 ('space', 0.0037248343498177943),
 ('baseball', 0.0032761865422322395),
 ('sale', 0.003185262403324349),
 ('encryption', 0.003098607267537597),
 ('god', 0.0030447237668035487),
 ('apple', 0.003014356422295267),
 ('clipper', 0.0028389522125914016),
 ('cars', 0.0028148847161224315),
 ('chip', 0.0026763448779193004),
 ('graphics', 0.002623344685929308),
 ('hockey', 0.002562255016443083),
 ('mac', 0.0024909256998940227),
 ('israeli', 0.002328820237287395),
 ('israel', 0.0021739060835089413),
 ('athos', 0.002129005426355418),
 ('atheists', 0.0020109180733767587),
 ('game', 0.001953336888706074),
 ('subject re', 0.0019332245924852422),
 ('gun', 0.0018551078387117567),
 ('he', 0.0018534074628202093),
 ('window', 0.0018264698001527876),
 ('team', 0.0017996249860857386),
 ('key', 0.0017787281595379558),
 ('guns', 0.001765019405